In [1]:
# Import statements for needed libraries
import requests
import json
import sys
import getopt
import time

In [2]:
# Enter a name for your network
new_admin_network = "ENTER A NETWORK NAME HERE"
# There are 4 devices in front of you; a MX Security Appliance, a MR AP, a MS switch, and an MV12 Camera.
# Enter the Serial numbers below to add them to the network
camera_serial = "PUT THE CAMERA SERIAL HERE"
ap_serial = "PUT THE API SERIAL HERE"
switch_serial = "PUT THE SWITCH SERIAL HERE"
sec_serial = "PUT THE MX SERIAL HERE"

In [3]:
# This is the Meraki API Key (DO NOT CHANGE!!!)
meraki_key = "b664c781ffc480e8bf35a9c582571f061eb5b474"

In [4]:
# This function creates the network
def create_network():
    # Head to https://developer.cisco.com/meraki/api/#/rest/api-endpoints/networks/create-organization-networks
    # to see the documentation to create a new network
    try:
        url = "https://api.meraki.com/api/v0/organizations/865776/networks"

        headers = {'X-Cisco-Meraki-API-Key': meraki_key,
                    'Content-Type':'application/json'}

        payload =   {
                        "name": new_admin_network,
                        "type": "wireless switch appliance camera"
                    }

        response = requests.request("POST", url, data=json.dumps(payload), \
        headers=headers)
   

        # Get the new network ID so we can add the devices
        network = json.loads(response.text)
        network_id = network["id"]
        print("This is your new network ID: " + network_id)

        
        serials = [sec_serial,ap_serial,switch_serial,camera_serial]

        for serial in serials:
            url = "https://api.meraki.com/api/v0/networks/"\
            +network_id+"/devices/claim"

            payload = {"serial": serial}
            headers = {'X-Cisco-Meraki-API-Key': meraki_key,
            'Content-Type': "application/json"}

            response = requests.request("POST", url, \
            data=json.dumps(payload), headers=headers)
            
            print(response.text)

        return network_id
    except Exception as e:
        print(e)

In [5]:
# Create two vlans, one for the MR AP and one for the MV camera
def create_vlans(network_id):
    print(network_id)
    url = "https://api.meraki.com/api/v0/networks/"+network_id+"/vlansEnabledState"
    
    payload = {"enabled": True}

    headers = {'X-Cisco-Meraki-API-Key': meraki_key,
    'Content-Type': "application/json"}

    try:
        response = requests.request("PUT", url, data=json.dumps(payload), \
        headers=headers)
        print(response.text)
    except Exception as e:
        print(e)
    # Head to https://developer.cisco.com/meraki/api/#/rest/api-endpoints/vlans/create-network-vlans
    # and use the documentation to create the URL to create a new VLAN (make sure to put the network_id in)
    url = "https://api.meraki.com/api/v0/networks/"+network_id+"/vlans"

    payload = {
      "id": "2",
      "name": "VLAN2",
      "subnet": "192.168.2.0/24",
      "applianceIp": "192.168.2.1"
    }

    headers = {'X-Cisco-Meraki-API-Key': meraki_key,
    'Content-Type': "application/json"}

    try:
        response = requests.request("POST", url, data=json.dumps(payload), \
        headers=headers)
        print(response.text)
        print("VLAN2 added")
    except Exception as e:
        print(e)
        
    payload = {
      "id": "3",
      "name": "VLAN3",
      "subnet": "192.168.3.0/24",
      "applianceIp": "192.168.3.1"
    }

    headers = {'X-Cisco-Meraki-API-Key': meraki_key,
    'Content-Type': "application/json"}

    try:
        response = requests.request("POST", url, data=json.dumps(payload), \
        headers=headers)
        print(response.text)
        print("VLAN3 added")
    except Exception as e:
        print(e)
        
    # Now set the switchports for the MR AP (port 2) and the MV Camera (port 3) to match the corresponding vlans.
    # Head to https://testing-developer.cisco.com/meraki/api/#/rest/api-endpoints/switch-ports/update-device-switch-port
    # and create the url that will change switchport 2 to vlan 2.
    url = "https://api.meraki.com/api/v0/devices/"+switch_serial+"/switchPorts/2"
    
    payload = {
        "vlan": "2",
        "type": "access"
    }

    headers = {'X-Cisco-Meraki-API-Key': meraki_key,
    'Content-Type': "application/json"}

    try:
        response = requests.request("PUT", url, data=json.dumps(payload), \
        headers=headers)
        print(response.text)
    except Exception as e:
        print(e)
        
    # Head to https://testing-developer.cisco.com/meraki/api/#/rest/api-endpoints/switch-ports/update-device-switch-port
    # and create the url that will change switchport 3 to vlan 3.
    url = "https://api.meraki.com/api/v0/devices/"+switch_serial+"/switchPorts/3"
    
    payload = {
        "vlan": "3",
        "type": "access"
    }

    headers = {'X-Cisco-Meraki-API-Key': meraki_key,
    'Content-Type': "application/json"}

    try:
        response = requests.request("PUT", url, data=json.dumps(payload), \
        headers=headers)
        print(response.text)
    except Exception as e:
        print(e)

In [6]:
# Create ssid for our MR to broadcast
def create_ssid(network_id):

    # Head to https://testing-developer.cisco.com/meraki/api/#/rest/api-endpoints/ssids/update-network-ssid
    # and use the documentation to create the URL to create a new SSID. Make sure to set the network_id.
    # The ssid number is 0
    url = "https://api.meraki.com/api/v0/networks/"+network_id+"/ssids/0"

    payload = {
        "id": 0,
        "name": new_admin_network,
        "enabled": True,
        "splashPage": "Click-through splash page",
        "ssidAdminAccessible": False,
        "authMode": "psk",
        "encryptionMode": "wpa",
        "psk": "devnetcreate",
        "ipAssignmentMode": "Bridge mode",
        "useVlanTagging": False,
        "minBitrate": 11,
        "bandSelection": "Dual band operation with Band Steering",
        "perClientBandwidthLimitUp": 0,
        "perClientBandwidthLimitDown": 0,
        "walledGardenEnabled": True,
        "walledGardenRanges": "*.amazonaws.com"
    }

    headers = {'X-Cisco-Meraki-API-Key': meraki_key,
    'Content-Type': "application/json"}

    try:
        response = requests.request("PUT", url, data=json.dumps(payload), \
        headers=headers)
        print("SSID CREATED")
    except Exception as e:
        print(e)

In [7]:
# Set Excap Splash Page
def splash_settings(network_id):

    # Head to https://testing-developer.cisco.com/meraki/api/#/rest/api-endpoints/splash-settings/update-network-ssids-plash-settings
    # and use the documentation to create the URL to set the splash page. Make sure to set the network_id.
    # The ssid number is 0
    url = "https://api.meraki.com/api/v0/networks/"+network_id+"/ssids/0/splashSettings"

    payload = {
        "splashUrl": "https://ietn805qd8.execute-api.us-east-1.amazonaws.com/dev/click?network_id="+network_id+"&camera_serial="+camera_serial+"&api_key="+meraki_key,
        "useSplashUrl": True
    }

    headers = {'X-Cisco-Meraki-API-Key': meraki_key,
    'Content-Type': "application/json"}

    try:
        response = requests.request("PUT", url, data=json.dumps(payload), \
        headers=headers)
        print("SPLASH SETTINGS SET")
        print("NOW CONNECT TO YOUR SSID ON YOUR MOBILE DEVICE")
    except Exception as e:
        print(e)

In [8]:
network_id = create_network()
time.sleep(2)
#create_vlans(network_id)
time.sleep(2)
create_ssid(network_id)
time.sleep(2)
splash_settings(network_id)


L_566327653141855376
 
 
 
 
{"number":0,"name":"Boomshakalaka","enabled":true,"splashPage":"Click-through splash page","ssidAdminAccessible":false,"authMode":"psk","psk":"devnetcreate","encryptionMode":"wpa","wpaEncryptionMode":"WPA2 only","ipAssignmentMode":"Bridge mode","useVlanTagging":false,"walledGardenEnabled":true,"walledGardenRanges":"*.amazonaws.com","minBitrate":11,"bandSelection":"Dual band operation with Band Steering","perClientBandwidthLimitUp":0,"perClientBandwidthLimitDown":0}
{"ssidNumber":0,"splashMethod":"Click-through splash page","splashUrl":"https://ietn805qd8.execute-api.us-east-1.amazonaws.com/dev/click?network_id=L_566327653141855376&camera_serial=Q2GV-Y4MD-VSQ6&api_key=b664c781ffc480e8bf35a9c582571f061eb5b474","useSplashUrl":true}
NOW CONNECT TO YOUR SSID ON YOUR MOBILE DEVICE


In [ ]:
#Now Cleanup and delete your Network that you just created...thanks!
# Get all devices in the network, remove them, delete the network, recreate
# the network and add devices back in.
def reset_network(network_id):
    try:
        # Get all network devices
        print "get networks devices"
        url = "https://api.meraki.com/api/v0/networks/"+network_id+"/devices"
        print url
        headers = {'X-Cisco-Meraki-API-Key': meraki_key}

        response = requests.request("GET", url,  headers=headers)
        print response.text

        # Remove devices from network
        for device in json.loads(response.text):
            print "delete all devices so we can re-add them"
            url = "https://api.meraki.com/api/v0/networks/"+network_id+"/devices"+device["serial"]+"/remove"

            headers = {'X-Cisco-Meraki-API-Key': meraki_key}

            response = requests.request("POST", url, headers=headers)
            print response.text

        # Delete the network
        print "delete the network"
        url = "https://api.meraki.com/api/v0/networks/"+network_id
        headers = {'X-Cisco-Meraki-API-Key': meraki_key}

        response = requests.request("DELETE", url, headers=headers)
        print response.text

In [ ]:
reset_network(network_id)